# 2-JoiningDatasets
This tutorial shows how to identify drug molecules in the PDB by joining two datasets: 

1. Drug information from DrugBank
2. Ligand information from RCSB PDB

In [1]:
from pyspark.sql import SparkSession
from mmtfPyspark.datasets import customReportService, drugBankDataset
from mmtfPyspark.structureViewer import view_binding_site

#### Configure Spark

In [2]:
spark = SparkSession.builder.appName("2-JoiningDatasets").getOrCreate()

## Download open DrugBank dataset
Download a dataset of drugs from [DrugBank](https://www.drugbank.ca) and filter out any drugs that do not have an InChIKey. [InChIKeys](https://en.wikipedia.org/wiki/International_Chemical_Identifier) are unique identifiers for small molecules. 

DrugBank provides more [detailed datasets](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/drugBankDataset.py), e.g., subset of approved drugs, but a DrugBank username and password is required. For this tutorial we use the open DrugBank dataset.

In [3]:
drugs = drugBankDataset.get_open_drug_links()
drugs = drugs.filter("StandardInChIKey IS NOT NULL").cache()
drugs.toPandas().head(5)

,DrugBankID,AccessionNumbers,Commonname,CAS,UNII,Synonyms,StandardInChIKey
0,DB00006,BIOD00076 | BTD00076 | DB02351 | EXPT03302,Bivalirudin,128270-60-0,TN9BEX005G,Bivalirudin | Bivalirudina | Bivalirudinum | H...,OIRCOABEOLEUMC-GEJPAHFPSA-N
1,DB00014,BIOD00113 | BTD00113,Goserelin,65807-02-5,0F65R8P09N,Goserelin | Goserelina,BLCLNMBMMGCOAS-URPVMXJPSA-N
2,DB00027,BIOD00036 | BTD00036,Gramicidin D,1405-97-6,5IE62321P4,Bacillus brevis gramicidin D | Gramicidin | Gr...,NDAYQJDHGXTBJL-MWWSRJDJSA-N
3,DB00035,BIOD00061 | BIOD00112 | BTD00061 | BTD00112,Desmopressin,16679-58-6,ENR1LLB0FP,1-(3-mercaptopropionic acid)-8-D-arginine-vaso...,NFLWUMRGJYTJIN-NXBWRCJVSA-N
4,DB00050,APRD00686 | BIOD00115 | BTD00115,Cetrorelix,120287-85-6,OON1HFZ4BA,Cetrorelix | Cetrorelixum,SBNPWPIBESPSIF-MHWMIDJBSA-N


## Download ligand annotations from RCSB PDB 
Here we use [RCSB PDB web services](http://dx.doi.org/10.1093/nar/gkq1021) to download InChIKeys and molecular weight for ligands in the PDB (this step can be slow!).

We filter out entries without an InChIKey and low molecular weight ligands using SQL syntax.

In [4]:
ligands = customReportService.get_dataset(["ligandId","InChIKey","ligandMolecularWeight"])

ligands = ligands.filter("InChIKey IS NOT NULL AND ligandMolecularWeight > 300").cache()

ligands.toPandas().head(10)

,structureChainId,structureId,chainId,ligandId,InChIKey,ligandMolecularWeight
0,101D.A,101D,A,CBR,PLDRCXOBLRYJSZ-RRKCRQDMSA-N,386.09
1,101D.B,101D,B,CBR,PLDRCXOBLRYJSZ-RRKCRQDMSA-N,386.09
2,101D.B,101D,B,NT,IDBIFFKSXLYUOT-UHFFFAOYSA-N,430.46
3,101M.A,101M,A,HEM,KABFMIBPWCXCRK-RGGAHWMASA-L,616.49
4,102D.B,102D,B,TNT,WTFXJFJYEJZMFO-UHFFFAOYSA-N,312.37
5,102M.A,102M,A,HEM,KABFMIBPWCXCRK-RGGAHWMASA-L,616.49
6,103M.A,103M,A,HEM,KABFMIBPWCXCRK-RGGAHWMASA-L,616.49
7,104M.A,104M,A,HEM,KABFMIBPWCXCRK-RGGAHWMASA-L,616.49
8,105M.A,105M,A,HEM,KABFMIBPWCXCRK-RGGAHWMASA-L,616.49
9,106M.A,106M,A,HEM,KABFMIBPWCXCRK-RGGAHWMASA-L,616.49


## Find drugs in PDB
By [joining](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=join#pyspark.sql.DataFrame.join) the two datasets on the InChIKey, we get the intersection between the two datasets.

In [5]:
ligands = ligands.join(drugs, ligands.InChIKey == drugs.StandardInChIKey)

#### Keep only unique ligands per structure

Here we [drop](https://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=join#pyspark.sql.DataFrame.dropDuplicates) rows with the same structureId and ligandId.

In [6]:
ligands = ligands.dropDuplicates(["structureId","ligandId"]).cache()

#### Keep only essential columns

In [7]:
ligands = ligands.select("structureId","ligandId","chainId","Commonname")
ligands.toPandas().head(10)

,structureId,ligandId,chainId,Commonname
0,1AJ7,NPE,H,5-(PARA-NITROPHENYL PHOSPHONATE)-PENTANOIC ACID
1,1B3R,NAD,A,Nadide
2,1EYR,CDP,A,Cytidine-5'-Diphosphate
3,1FBZ,CC1,A,{4-[2-Acetylamino-2-(3-Carbamoyl-2-Cyclohexylm...
4,1G17,GNP,A,Phosphoaminophosphonic Acid Guanylate Ester
5,1G1L,DAU,A,2'deoxy-Thymidine-5'-Diphospho-Alpha-D-Glucose
6,1H1W,ATP,A,ATP
7,1H86,FAD,A,Flavin adenine dinucleotide
8,1HNB,GDN,A,"S-(2,4-dinitrophenyl)glutathione"
9,1HTT,AMP,A,Adenosine phosphate


## Visualize drug binding sites

#### Extract id columns as lists (required for visualization)

In [8]:
pdb_ids = ligands.select("structureId").rdd.flatMap(lambda x: x).collect()
ligand_ids = ligands.select("ligandId").rdd.flatMap(lambda x: x).collect()
chain_ids = ligands.select("chainId").rdd.flatMap(lambda x: x).collect()

Disable scrollbar for the visualization below

In [9]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

<IPython.core.display.Javascript object>

#### Show binding site residues within 4.5 A from the drug molecule

In [10]:
view_binding_site(pdb_ids, ligand_ids, chain_ids, distance=4.5);

interactive(children=(IntSlider(value=0, continuous_update=False, description='Structure', max=29915), Output(…

In [11]:
spark.stop()